# Clasificación de Piso en el Dataset UJIIndoorLoc

---

## Introducción

En este notebook se implementa un flujo completo de procesamiento y análisis para la clasificación del **piso** en un entorno interior utilizando el dataset **UJIIndoorLoc**. Este conjunto de datos contiene mediciones de señales WiFi recopiladas en distintas ubicaciones de un edificio, con información sobre coordenadas, piso, usuario, hora, entre otros.

En esta tarea nos enfocaremos en predecir el **piso** en el que se encuentra un dispositivo, considerando únicamente las muestras etiquetadas con valores válidos para dicha variable. Se tratará como un problema de clasificación multiclase (planta baja, primer piso, segundo piso).

## Objetivos

- **Cargar y explorar** el conjunto de datos UJIIndoorLoc.
- **Preparar** los datos seleccionando las características relevantes y el target (`FLOOR`).
- **Dividir** el dataset en entrenamiento y validación (80/20).
- **Entrenar y optimizar** clasificadores basados en seis algoritmos:
  - K-Nearest Neighbors (KNN)
  - Gaussian Naive Bayes
  - Regresión Logística
  - Árboles de Decisión
  - Support Vector Machines (SVM)
  - Random Forest
- **Seleccionar hiperparámetros óptimos** para cada modelo utilizando validación cruzada (5-fold), empleando estrategias como **Grid Search**, **Randomized Search**, o **Bayesian Optimization** según el algoritmo.
- **Comparar el desempeño** de los modelos sobre el conjunto de validación, usando métricas como *accuracy*, *precision*, *recall*, y *F1-score*.
- **Determinar el mejor clasificador** para esta tarea, junto con sus hiperparámetros óptimos.

Este ejercicio permite no solo evaluar la capacidad predictiva de distintos algoritmos clásicos de clasificación, sino también desarrollar buenas prácticas en validación de modelos y selección de hiperparámetros en contextos del mundo real.

---

## Descripción del Dataset

El dataset utilizado en este análisis es el **UJIIndoorLoc Dataset**, ampliamente utilizado para tareas de localización en interiores a partir de señales WiFi. Está disponible públicamente en la UCI Machine Learning Repository y ha sido recopilado en un entorno real de un edificio universitario.

Cada muestra corresponde a una observación realizada por un dispositivo móvil, donde se registran las intensidades de señal (RSSI) de más de 500 puntos de acceso WiFi disponibles en el entorno. Además, cada fila contiene información contextual como la ubicación real del dispositivo (coordenadas X e Y), el piso, el edificio, el identificador del usuario, y la marca temporal.

El objetivo en esta tarea es predecir el **piso** (`FLOOR`) en el que se encontraba el dispositivo en el momento de la medición, considerando únicamente las características numéricas provenientes de las señales WiFi.

### Estructura del dataset

- **Número de muestras**: ~20,000
- **Número de características**: 520
  - 520 columnas con valores de intensidad de señal WiFi (`WAP001` a `WAP520`)
- **Variable objetivo**: `FLOOR` (variable categórica con múltiples clases, usualmente entre 0 y 4)

### Columnas relevantes

- `WAP001`, `WAP002`, ..., `WAP520`: niveles de señal recibida desde cada punto de acceso WiFi (valores entre -104 y 0, o 100 si no se detectó).
- `FLOOR`: clase objetivo a predecir (nivel del edificio).
- (Otras columnas como `BUILDINGID`, `SPACEID`, `USERID`, `TIMESTAMP`, etc., pueden ser ignoradas o utilizadas en análisis complementarios).

### Contexto del problema

La localización en interiores es un problema complejo en el que tecnologías como el GPS no funcionan adecuadamente. Los sistemas basados en WiFi han demostrado ser una alternativa efectiva para estimar la ubicación de usuarios en edificios. Poder predecir automáticamente el piso en el que se encuentra una persona puede mejorar aplicaciones de navegación en interiores, accesibilidad, gestión de emergencias y servicios personalizados. Este tipo de problemas es típicamente abordado mediante algoritmos de clasificación multiclase.


### Estrategia de evaluación

En este análisis seguiremos una metodología rigurosa para garantizar la validez de los resultados:

1. **Dataset de entrenamiento**: Se utilizará exclusivamente para el desarrollo, entrenamiento y optimización de hiperparámetros de todos los modelos. Este conjunto será dividido internamente en subconjuntos de entrenamiento y validación (80/20) para la selección de hiperparámetros mediante validación cruzada.

2. **Dataset de prueba**: Se reservará únicamente para la **evaluación final** de los modelos ya optimizados. Este conjunto **no debe ser utilizado** durante el proceso de selección de hiperparámetros, ajuste de modelos o toma de decisiones sobre la arquitectura, ya que esto introduciría sesgo y comprometería la capacidad de generalización estimada.

3. **Validación cruzada**: Para la optimización de hiperparámetros se empleará validación cruzada 5-fold sobre el conjunto de entrenamiento, lo que permitirá una estimación robusta del rendimiento sin contaminar los datos de prueba.

Esta separación estricta entre datos de desarrollo y evaluación final es fundamental para obtener una estimación realista del rendimiento que los modelos tendrían en un escenario de producción con datos completamente nuevos.

---


## Paso 1: Cargar y explorar el dataset

**Instrucciones:**
- Descarga el dataset **UJIIndoorLoc** desde la UCI Machine Learning Repository o utiliza la versión proporcionada en el repositorio del curso (por ejemplo: `datasets\UJIIndoorLoc\trainingData.csv`).
- Carga el dataset utilizando `pandas`.
- Muestra las primeras filas del dataset utilizando `df.head()`.
- Imprime el número total de muestras (filas) y características (columnas).
- Verifica cuántas clases distintas hay en la variable objetivo `FLOOR` y cuántas muestras tiene cada clase (`df['FLOOR'].value_counts()`).


In [ ]:
# tu código aquí

---

## Paso 2: Preparar los datos

**Instrucciones:**

- Elimina las columnas que no son relevantes para la tarea de clasificación del piso:
  - `LONGITUDE`, `LATITUDE`, `SPACEID`, `RELATIVEPOSITION`, `USERID`, `PHONEID`, `TIMESTAMP`
- Conserva únicamente:
  - Las columnas `WAP001` a `WAP520` como características (RSSI de puntos de acceso WiFi).
  - La columna `FLOOR` como variable objetivo.
- Verifica si existen valores atípicos o valores inválidos en las señales WiFi (por ejemplo: valores constantes como 100 o -110 que suelen indicar ausencia de señal).
- Separa el conjunto de datos en:
  - `X`: matriz de características (todas las columnas `WAP`)
  - `y`: vector objetivo (`FLOOR`)


In [ ]:
# tu código aquí

--- 

## Paso 3: Preprocesamiento de las señales WiFi

**Contexto:**

Las columnas `WAP001` a `WAP520` representan la intensidad de la señal (RSSI) recibida desde distintos puntos de acceso WiFi. Los valores típicos de RSSI están en una escala negativa, donde:

- Valores cercanos a **0 dBm** indican señal fuerte.
- Valores cercanos a **-100 dBm** indican señal débil o casi ausente.
- Un valor de **100** en este dataset representa una señal **no detectada**, es decir, el punto de acceso no fue visto por el dispositivo en ese instante.

**Instrucciones:**

- Para facilitar el procesamiento y tratar la ausencia de señal de forma coherente, se recomienda mapear todos los valores **100** a **-100**, que semánticamente representa *ausencia de señal detectable*.
- Esto unifica el rango de valores y evita que 100 (un valor artificial) afecte negativamente la escala de los algoritmos.

**Pasos sugeridos:**

- Reemplaza todos los valores `100` por `-100` en las columnas `WAP001` a `WAP520`:
  ```python
  X[X == 100] = -100


In [ ]:
# tu código aquí

--- 

## Paso 4: Entrenamiento y optimización de hiperparámetros

**Objetivo:**

Entrenar y comparar distintos clasificadores para predecir correctamente el piso (`FLOOR`) y encontrar los mejores hiperparámetros para cada uno mediante validación cruzada.

**Clasificadores a evaluar:**

- K-Nearest Neighbors (KNN)
- Gaussian Naive Bayes
- Regresión Logística
- Árboles de Decisión
- Support Vector Machines (SVM)
- Random Forest

**Procedimiento:**

1. Divide el dataset en conjunto de **entrenamiento** (80%) y **validación** (20%) usando `train_test_split` con `stratify=y`.
2. Para cada clasificador:
   - Define el espacio de búsqueda de hiperparámetros.
   - Usa **validación cruzada 5-fold** sobre el conjunto de entrenamiento para seleccionar los mejores hiperparámetros.
   - Emplea una estrategia de búsqueda adecuada:
     - **GridSearchCV**: búsqueda exhaustiva (ideal para espacios pequeños).
     - **RandomizedSearchCV**: búsqueda aleatoria (más eficiente con espacios amplios).
     - **Bayesian Optimization** (opcional): para búsquedas más inteligentes, usando librerías como `optuna` o `skopt`.
3. Guarda el mejor modelo encontrado para cada clasificador con su configuración óptima.



In [ ]:
# create the training and validation sets

In [ ]:
# train and optimize KNN

In [ ]:
# train and optimize Gaussian Naive Bayes

In [ ]:
# train and optimize Logistic Regression

In [ ]:
# train and optimize decision tree

In [ ]:
# train and optimize Support Vector Machine

In [ ]:
# train and optimize Random Forest

---

## Paso 5: Crear una tabla resumen de los mejores modelos

**Instrucciones:**

Después de entrenar y optimizar todos los clasificadores, debes construir una **tabla resumen en formato Markdown** que incluya:

- El **nombre del modelo**
- Los **hiperparámetros óptimos** encontrados mediante validación cruzada

### Requisitos:

- La tabla debe estar escrita en formato **Markdown**.
- Cada fila debe corresponder a uno de los modelos evaluados.
- Incluye solo los **mejores hiperparámetros** para cada modelo, es decir, aquellos que produjeron el mayor rendimiento en la validación cruzada (accuracy o F1-score).
- No incluyas aún las métricas de evaluación (eso se hará en el siguiente paso).

### Ejemplo de formato:


| Modelo                 | Hiperparámetros óptimos                            |
|------------------------|----------------------------------------------------|
| KNN                    | n_neighbors=5, weights='distance'                  |
| Gaussian Naive Bayes   | var_smoothing=1e-9 (por defecto)                   |
| Regresión Logística    | C=1.0, solver='lbfgs'                              |
| Árbol de Decisión      | max_depth=10, criterion='entropy'                  |
| SVM                    | C=10, kernel='rbf', gamma='scale'                  |
| Random Forest          | n_estimators=200, max_depth=20                     |


# tu tabla de resultados aquí

---

## Paso 6: Preparar los datos finales para evaluación

**Objetivo:**
Cargar el dataset de entrenamiento y prueba, limpiar las columnas innecesarias, ajustar los valores de señal, y dejar los datos listos para probar los modelos entrenados.

**Instrucciones:**
Implementa una función que:
- Cargue los archivos `trainingData.csv` y `validationData.csv`
- Elimine las columnas irrelevantes (`LONGITUDE`, `LATITUDE`, `SPACEID`, `RELATIVEPOSITION`, `USERID`, `PHONEID`, `TIMESTAMP`)
- Reemplace los valores `100` por `-100` en las columnas `WAP001` a `WAP520`
- Separe las características (`X`) y la variable objetivo (`FLOOR`)
- Devuelva los conjuntos `X_train`, `X_test`, `y_train`, `y_test`

In [ ]:
# tu código aquí

---

## Paso 7: Evaluar modelos optimizados en el conjunto de prueba

**Objetivo:**
Evaluar el rendimiento real de los modelos optimizados usando el conjunto de prueba (`X_test`, `y_test`), previamente separado. Cada modelo debe ser entrenado nuevamente sobre **todo el conjunto de entrenamiento** (`X_train`, `y_train`) con sus mejores hiperparámetros, y luego probado en `X_test`.

**Instrucciones:**

1. Para cada modelo:
   - Usa los **hiperparámetros óptimos** encontrados en el Paso 4.
   - Entrena el modelo con `X_train` y `y_train`.
   - Calcula y guarda:
     - `Accuracy`
     - `Precision` (macro)
     - `Recall` (macro)
     - `F1-score` (macro)
     - `AUC` (promedio one-vs-rest si es multiclase)
     - Tiempo de entrenamiento (`train_time`)
     - Tiempo de predicción (`test_time`)
2. Muestra todos los resultados en una **tabla comparativa**


In [ ]:
# tu código aquí

---
## Paso 8: Selección y justificación del mejor modelo

**Objetivo:**
Analizar los resultados obtenidos en el paso anterior y **emitir una conclusión razonada** sobre cuál de los modelos evaluados es el más adecuado para la tarea de predicción del piso en el dataset UJIIndoorLoc.

**Instrucciones:**

- Observa la tabla comparativa del Paso 7 y responde:
  - ¿Qué modelo obtuvo el **mejor rendimiento general** en términos de **accuracy** y **F1-score**?
  - ¿Qué tan consistente fue su rendimiento en **precision** y **recall**?
  - ¿Tiene un **tiempo de entrenamiento o inferencia** excesivamente alto?
  - ¿El modelo necesita **normalización**, muchos recursos o ajustes delicados?
- Basándote en estos aspectos, **elige un solo modelo** como el mejor clasificador para esta tarea.
- **Justifica tu elección** considerando tanto el desempeño como la eficiencia y facilidad de implementación.


# tu respuesta aquí

---

## Rúbrica de Evaluación

| Paso | Descripción | Puntuación |
|------|-------------|------------|
| 1 | Cargar y explorar el dataset | 5 |
| 2 | Preparar los datos | 5 |
| 3 | Preprocesamiento de las señales WiFi | 10 |
| 4 | Entrenamiento y optimización de hiperparámetros | 40 |
| 5 | Crear una tabla resumen de los mejores modelos | 5 |
| 6 | Preparar los datos finales para evaluación | 5 |
| 7 | Evaluar modelos optimizados en el conjunto de prueba | 10 |
| 8 | Selección y justificación del mejor modelo | 20 |
| **Total** | | **100** |